In [ ]:
import logging
from pyspark.sql import functions as F

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Function to load data from MySQL
def load_mysql_data(query, secret_scope, secret_key):
    try:
        # Retrieve credentials securely
        user = dbutils.secrets.get(secret_scope, f"{secret_key}_user")
        password = dbutils.secrets.get(secret_scope, f"{secret_key}_password")
        host = dbutils.secrets.get(secret_scope, f"{secret_key}_host")
        database = dbutils.secrets.get(secret_scope, f"{secret_key}_database")
        
        # Connect to MySQL and fetch data using JDBC
        jdbc_url = f"jdbc:mysql://{host}/{database}"
        df = spark.read.format("jdbc").option("url", jdbc_url) \
            .option("dbtable", f"({query}) as t").option("user", user).option("password", password).load()
        
        logger.info(f"Loaded data from MySQL with query: {query}")
        logger.info(f"Schema: {df.schema}")
        logger.info(f"Record count: {df.count()}")
        return df
    except Exception as e:
        logger.error(f"Error loading data from MySQL: {str(e)}")
        raise

# Function to load data from Unity Catalog
def load_unity_catalog_data(table_path):
    try:
        # Correct the table path syntax for Unity Catalog
        df = spark.table(table_path)
        logger.info(f"Loaded data from Unity Catalog table: {table_path}")
        logger.info(f"Schema: {df.schema}")
        logger.info(f"Record count: {df.count()}")
        return df
    except Exception as e:
        logger.error(f"Error loading data from Unity Catalog: {str(e)}")
        raise

# Function to apply transformations
def apply_transformations(df):
    try:
        # Example transformation: Clean and transform multiple fields
        df_transformed = df.withColumn("field", F.when(F.col("field").contains("#"), None)
                                        .when(F.col("field").contains("UNMAPPED"), None)
                                        .when(F.col("field").contains("NULL"), None)
                                        .otherwise(F.col("field"))) \
                           .withColumn("field", F.when(F.col("field").startswith("00"), F.expr("substring(field, 3, length(field))"))
                                        .otherwise(F.col("field")))
        
        logger.info("Applied transformations")
        logger.info(f"Schema after transformation: {df_transformed.schema}")
        logger.info(f"Record count after transformation: {df_transformed.count()}")
        return df_transformed
    except Exception as e:
        logger.error(f"Error applying transformations: {str(e)}")
        raise

# Function to perform join operations
def perform_joins(df1, df2, join_condition):
    try:
        df_joined = df1.join(df2, join_condition, "inner")
        logger.info("Performed join operation")
        logger.info(f"Schema after join: {df_joined.schema}")
        logger.info(f"Record count after join: {df_joined.count()}")
        return df_joined
    except Exception as e:
        logger.error(f"Error performing join operation: {str(e)}")
        raise

# Function to write data to Unity Catalog
def write_to_unity_catalog(df, catalog, schema, table):
    try:
        # Ensure schema exists
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")
        logger.info(f"Schema {catalog}.{schema} ensured")
        
        # Write data to Unity Catalog table
        df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{table}")
        logger.info(f"Data written to Unity Catalog table: {catalog}.{schema}.{table}")
    except Exception as e:
        logger.error(f"Error writing data to Unity Catalog: {str(e)}")
        raise

# Load data from various sources
mysql_df_99 = None
text_input_df_330 = None
try:
    mysql_df_99 = load_mysql_data("select pjotr_prod.* from pjotr_prod", "mysql_scope", "pjotr_dev")
except Exception as e:
    logger.error(f"Failed to load MySQL data for Node 99: {str(e)}")

try:
    # Correct the table path syntax for Unity Catalog
    text_input_df_330 = load_unity_catalog_data("catalog.schema.TextInput")
except Exception as e:
    logger.error(f"Failed to load Unity Catalog data for Node 330: {str(e)}")

# Apply transformations
transformed_df = None
if mysql_df_99 is not None:
    try:
        transformed_df = apply_transformations(mysql_df_99)
    except Exception as e:
        logger.error(f"Failed to apply transformations: {str(e)}")

# Perform join operations
joined_df = None
if transformed_df is not None and text_input_df_330 is not None:
    try:
        joined_df = perform_joins(transformed_df, text_input_df_330, "join_field")
    except Exception as e:
        logger.error(f"Failed to perform join operations: {str(e)}")

# Write output to Unity Catalog
if joined_df is not None:
    try:
        write_to_unity_catalog(joined_df, "catalog_name", "schema_name", "C03_pjotr")
    except Exception as e:
        logger.error(f"Failed to write data to Unity Catalog: {str(e)}")

# Corrected code to handle secret and table access issues
# 1. Ensure the secret scope and keys are correctly configured in Databricks.
# 2. Verify the Unity Catalog table path is correct and accessible.

# Additional logging and error handling added to ensure robustness in Databricks environment.
